In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [ ]:
!pip install GitPython
!pip install --force https://github.com/chengs/tqdm/archive/colab.zip

In [ ]:
import git
import subprocess
import shutil
import os
import sys

class Yolo:

  def __init__(self,working_dir=None):
    self.cwd=working_dir if working_dir != None else "/content"
  
  @staticmethod
  def import_model(cwd):
    
    url="https://github.com/ultralytics/yolov5"
    repo_name=os.path.basename(url)
    path=os.path.join(cwd,repo_name) 
    print(path)
    
    if  not os.path.exists(path):
      git.Git(cwd).clone(url)
    
    else:    
      user_input=input("Non_Empty directory, replace? press y for Yes, n for No:")
      if user_input=="y":
        shutil.rmtree(path)
        git.Git(cwd).clone(url)
 
    os.chdir("/content/yolov5")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])
    os.chdir("/content")
    print("Requrements installed.")
  
  
cwd="/content/"
Yolo().import_model(cwd)

In [ ]:
import os
path_train_images=os.path.normpath("/content/drive/My Drive/Bath thesis/Final-Data/Augmentation Mosaic (3,4)/train_images")
path_valid_images=os.path.normpath("/content/drive/My Drive/Bath thesis/Final-Data/Not Augmented/valid_images")

num_train=sum([1 for i in os.listdir(path_train_images)])
num_valid=sum([1 for i in os.listdir(path_valid_images)])
print("There are {} train and {} valid images.".format(num_train,num_valid))

def write_yaml(path_train,path_valid):

  num_classes=sum([1 for file in open("/content/drive/My Drive/Bath thesis/Final-Data/Not Augmented/classes.txt")])
  label_list=([str(label).strip("\n") for label in open("/content/drive/My Drive/Bath thesis/Final-Data/Not Augmented/classes.txt")])

  with open(os.path.join("/content","data.yaml"),"wt") as file:

    file.write("train: "+path_train_images+"\n")
    file.write("val: "+path_valid_images+"\n")
    file.write("\n")
    file.write("nc: "+str(num_classes)+"\n")
    file.write("names: "+str(label_list))

write_yaml(path_train_images,path_valid_images)


There are 1400 train and 200 valid images.


## Directory note

*   When specifying train_images, _images is replaces with _labels and  the labels are found, so make sure the directories are listed as such.

*   An alternative is to have train/images and train/labels, valid/images and valid/labels or images/train,images/valid and labels/train,labels/valid. They key part is when the images are replaced with labels, the right path is found.



In [ ]:
import git
url="https://github.com/karen-gishyan/Computer-vision-and-Drones-Thesis"
try:
  git.Git().clone(url)
except Exception as e:
  print(e)
finally:
  shutil.rmtree(os.path.split(url)[1])
  git.Git().clone(url)
  print("Done")

Done


In [ ]:
#Copying custom config and train to train environment (no augmentation and modified classes).
import shutil
shutil.copy("/content/Computer-vision-and-Drones-Thesis/models/Yolo Config/yolov5l_custom.yaml","/content/yolov5/models")
shutil.copy("/content/Computer-vision-and-Drones-Thesis/models/Yolo Config/yolov5_train_custom.py","/content/yolov5")

In [ ]:
#Downloads automatically pretrained yolov5m and we specify yolov5m.yaml.
%%time
%cd /content/yolov5/

!python yolov5_train_custom.py --img 1056 --batch 2 --epochs 20 --data '../data.yaml' --cfg yolov5l_custom.yaml --weights yolov5l.pt --name yolov5l_results  #--nosave --cache


In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/yolov5/runs # if valid loss is high, even when the model learns, there is some overfitting issue.

In [ ]:
#Tensorboard output to csv.
#https://stackoverflow.com/questions/42355122/can-i-export-a-tensorflow-summary-to-csv.

import os
import numpy as np
import pandas as pd

from collections import defaultdict
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator


def tabulate_events(dpath):
    summary_iterators = [EventAccumulator(os.path.join(dpath, dname)).Reload() for dname in os.listdir(dpath)]

    tags = summary_iterators[0].Tags()['scalars']

    for it in summary_iterators:
        assert it.Tags()['scalars'] == tags

    out = defaultdict(list)
    steps = []

    for tag in tags:
        steps = [e.step for e in summary_iterators[0].Scalars(tag)]

        for events in zip(*[acc.Scalars(tag) for acc in summary_iterators]):
            assert len(set(e.step for e in events)) == 1

            out[tag].append([e.value for e in events])

    return out, steps


def to_csv(dpath):
    dirs = os.listdir(dpath)

    d, steps = tabulate_events(dpath)
    tags, values = zip(*d.items())
    np_values = np.array(values)

    for index, tag in enumerate(tags):
        df = pd.DataFrame(np_values[index], index=steps, columns=dirs)
        df.to_csv(get_file_path(dpath, tag))


def get_file_path(dpath, tag):
    file_name = tag.replace("/", "_") + '.csv'
    folder_path = os.path.join(dpath, 'csv')
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    return os.path.join(folder_path, file_name)


if __name__ == '__main__':
    path = "/content/yolov5/runs"
    to_csv(path)

In [ ]:
shutil.copy("/content/yolov5/runs/exp2_yolov5l_results/weights/yolov5_best with no def aug.pt","/content/drive/My Drive/Bath Thesis/Augmentation Mosaic (3,4)")

In [ ]:
#Downloading runs and tesnorflow results..

#!zip -r /content/yolov5l_results.zip /content/yolov5/runs/exp0_yolov5l_results
!zip -r /content/yolov5l_results_csv.zip /content/yolov5/runs/csv

# Copy weights
# import shutil
#shutil.copytree("/content/yolov5/runs/exp2_yolov5l_results","/content/drive/My Drive/Bath Thesis/Augmentation 3 new new/yolov5 resultst")

In [ ]:
import shutil
shutil.copytree("/content/drive/My Drive/Bath thesis/Final-Data/Not Augmented/test_images","/content/test_images/images")
#shutil.copy("/content/yolov5/runs/exp0_yolov5l_results/weights/best.pt","/content/drive/My Drive/Bath Thesis/Augmentation Mosaic (3,4)")


'/content/test_images/images'

In [ ]:
import shutil
shutil.copy("/content/drive/My Drive/Bath thesis/Final-Data/Augmentation (3,5) and Blur/trainsfer_learning.pt","/content")

'/content/trainsfer_learning.pt'

In [ ]:
#Predicting batch of images. For the video, specify the video-path. ,--conf-thres 0.7.
!python detect.py --weights /content/trainsfer_learning.pt --source /content/test_images/images --img-size 1056
#!python detect.py --weights /content/yolov5/runs/exp0_yolov5l_results/weights/best_yolov5l_results.pt --source /content/images3 


In [ ]:
#Zipping predicted images for download.
!zip -r /content/yolov5l_inference.zip /content/yolov5/inference/output